# Linear global sensitivity analysis

When we don't include correlations, our systems are close to linear, and we can use simple correlation coefficients.

## Basic setup

In [28]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw2analyzer as ba
import numpy as np
from tqdm import tqdm
from scipy.stats import spearmanr
import math

In [2]:
if 'ei38-teaching-25' not in bd.projects:
    bi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")

In [3]:
bd.projects.set_current('ei38-teaching-25')

In [4]:
gwp = ('IPCC 2013', 'climate change', 'GWP 100a')

In [9]:
act = bd.Database("ei 3.8 cutoff").random()
act

'electricity production, oil' (kilowatt hour, CN-ZJ, None)

In [20]:
act.id

20788

Set up `LCA` object

In [10]:
lca = bc.LCA({act: 1}, gwp, use_distributions=True)
lca.lci()
lca.lcia()
lca.score

1.0437060736767432

We have 240k parameters, of which ~200k are uncertain.

In [11]:
lca.technosphere_mm.input_data_vector().shape

(241507,)

The row and column indices of the non-zero values in the technosphere:

In [12]:
lca.technosphere_mm.input_row_col_indices().shape

(241507,)

Generate an array of sampled values, and a vector of LCA scores

In [13]:
def draw_samples(lca, samples=25):
    data = np.zeros((samples, len(lca.technosphere_mm.input_data_vector())))
    scores = np.zeros((samples,))
    
    for i, _ in tqdm(zip(range(samples), lca)):
        data[i, :] = lca.technosphere_mm.input_data_vector()
        scores[i] = lca.score
        
    return data, scores

In [14]:
samples, scores = draw_samples(lca)

25it [00:18,  1.34it/s]


Calculate Spearman rank order correlation between sample values and LCA scores.

Uses a chunked algorithm so our servers don't combust.

In [15]:
def get_spearman_corr_coefficients(samples, scores, chunk_size=100):
    all_same_mask = (np.diff(samples.T).sum(axis=1) != 0).T
    samples_masked = samples[:, all_same_mask]
    chunks = math.ceil(samples_masked.shape[1] / chunk_size)

    corr = np.zeros_like(all_same_mask, dtype=float)
    corr_masked = corr[all_same_mask]
    
    for i in tqdm(range(chunks)):
        corr_masked[i * chunk_size:(i + 1) * chunk_size] = spearmanr(
            samples_masked[:, i * chunk_size:(i + 1) * chunk_size], 
            scores
        )[0][:-1, -1]
    
    corr[all_same_mask] = corr_masked
    return corr

In [16]:
corr = get_spearman_corr_coefficients(samples, scores)

100%|██████████| 1913/1913 [00:30<00:00, 62.49it/s]


Display correlations in human-readable form

In [17]:
def get_sensitive_exchanges(corr, lca, cutoff=25):
    top_elems = lca.technosphere_mm.input_row_col_indices()[np.argsort(corr)][:cutoff]
    top_elems_mapped = [(lca.dicts.product.reversed[x], lca.dicts.activity.reversed[y]) for x, y in top_elems]
    
    return [
        (
            score,
            # lca.technosphere_matrix[indices[0], indices[1]],
            bd.get_node(id=row[0]),
            bd.get_node(id=row[1]),
            indices,
            # row,
        )
        for row, indices, score in zip(top_elems_mapped, top_elems, sorted(corr))
    ]

In [18]:
get_sensitive_exchanges(corr, lca, 10)

[(-0.7992307692307692,
  'market for blasting' (kilogram, GLO, None),
  'tungsten mine operation and beneficiation' (kilogram, RoW, None),
  (1557, 15323)),
 (-0.7584615384615384,
  'market for waste wood, untreated' (kilogram, MK, None),
  'orange production, fresh grade' (kilogram, RoW, None),
  (18291, 8902)),
 (-0.7576923076923077,
  'market for synthetic rubber' (kilogram, GLO, None),
  'room-connecting overflow element production, steel, approx. 40 m3/h' (unit, RER, None),
  (6104, 5560)),
 (-0.7569230769230769,
  'market for electricity, low voltage' (kilowatt hour, AZ, None),
  'water pump operation, electric' (megajoule, RoW, None),
  (9053, 11143)),
 (-0.7492307692307693,
  'market for transport, freight, inland waterways, barge' (ton kilometer, RoW, None),
  'market for ferrous metal, in mixed metal scrap' (kilogram, RoW, None),
  (5, 16813)),
 (-0.7415384615384616,
  'market for waste mineral oil' (kilogram, Europe without Switzerland, None),
  'heat and power co-generation

In [40]:
ba.print_recursive_calculation(act, gwp, max_level=4, cutoff=1e-4)

Fraction of score | Absolute score | Amount | Activity
0001 | 0.858 |     1 | 'electricity production, oil' (kilowatt hour, CN-ZJ, None)
  0.000161 | 0.0001379 | -0.0001442 | 'market for municipal solid waste' (kilogram, RoW, None)
  0.00226 | 0.001937 | 1.184e-11 | 'market for oil power plant, 500MW' (unit, GLO, None)
    0.00027 | 0.0002314 | 1.588e-12 | 'oil power plant construction, 500MW' (unit, RER, None)
      0.000134 | 0.0001151 | 5.24e-05 | 'market for reinforcing steel' (kilogram, GLO, None)
    0.00199 | 0.001706 | 1.025e-11 | 'oil power plant construction, 500MW' (unit, RoW, None)
      0.000866 | 0.0007427 | 0.0003382 | 'market for reinforcing steel' (kilogram, GLO, None)
        0.000266 | 0.0002281 | 0.0001075 | 'reinforcing steel production' (kilogram, Europe without Austria, None
        0.000582 | 0.0004997 | 0.0002286 | 'reinforcing steel production' (kilogram, RoW, None)
      0.000298 | 0.0002554 | 0.002767 | 'market for diesel, burned in building machine' (megajo